# Preparing Data 4 the AE

### Import and function definition

In [1]:
import sys, os
sys.path.append(os.path.abspath('./oms-api-client'))

In [2]:
import pandas as pd
import numpy as np
import timeit, math, copy, yaml, ast
from omsapi import OMSAPI
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import curve_fit
import re
from drawing_utilities import *
#%load_ext memory_profiler

In [3]:
try:
    import runregistry
except:
    !pip install runregistry
    #IMPORTANT: change this path based on the output of
    #!pip show runregistry
    sys.path.append('/eos/home-i03/m/mcrucian/.local/lib/python3.9/site-packages')
    import runregistry

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: gosam 2.1.1-4b98559 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gosam or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: professor2 X.Y.Z has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of professor2 or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


### Read data and add lumi info

In [4]:
path = "/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs"
dirs = os.listdir(path)

# Regex pattern to capture the part up to the number
pattern = r'hRHGlobal[mp]\d+'

unique_matches = sorted({match for dir_name in os.listdir(path) for match in re.findall(pattern, dir_name)})

# Display the unique matches
print("Available MEs in dir:")
print('\n'.join(unique_matches))

Available MEs in dir:
hRHGlobalm2
hRHGlobalm3
hRHGlobalm4
hRHGlobalp2
hRHGlobalp3
hRHGlobalp4


In [7]:
out_label = "180924"
me = "hRHGlobalm2"
dirs = os.listdir(path)
me_dirs = [os.path.join(path, i) for i in dirs if me in i and os.path.isdir(os.path.join(path, i))]
print(me_dirs)

['/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D', '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2E', '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2F', '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2G']


In [8]:
files_all = []
for dir in me_dirs:
    files = os.listdir(dir)
    me_files = [dir+"/"+i for i in files if me in i]
    files_all = files_all + me_files
filtered_files = [file for file in files_all if os.path.exists(file) and os.path.getsize(file) >= 601]
filtered_files

['/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380253_380287.parquet',
 '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380288_380322.parquet',
 '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380323_380357.parquet',
 '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380358_380392.parquet',
 '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380393_380427.parquet',
 '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380428_380462.parquet',
 '/eos/cms/store/group/ml/AD4MVDHackathon/ML4DQM_MUON/MEs/hRHGlobalm2D/CSC_CSCOfflineMonitor_recHits_hRHGlobalm2_None_380463_380497.parquet',
 '/eos

In [ ]:
%%time
#monitoring_elements =pl.read_parquet(filtered_files).filter((pl.col('dataset').str.contains("StreamExpress"))).to_pandas()
monitoring_elements = pd.read_parquet(filtered_files)
monitoring_elements = monitoring_elements[monitoring_elements['dataset'].str.contains("StreamExpress")]

In [ ]:
monitoring_elements

In [ ]:
dataset = monitoring_elements["dataset"].unique()
print(dataset)

In [ ]:
run_list = np.sort(np.unique(monitoring_elements["run_number"].unique()))
print(run_list)

In [ ]:
#if you get errors here, go through the readme and look for the instructions on how to get runregistry credentials
runreg_df = pd.DataFrame(columns=["run_number", "cscGOOD", "cscSTANDBY", "cscBAD", "cscEMPTY"])
bad_runs = []
for r in run_list:
    run = runregistry.get_run(run_number=int(r))
    try:
        dict = {"run_number": int(r), "class": run["class"], "cscGOOD": 0, "cscSTANDBY":0, "cscBAD":0, "cscEMPTY":0}
        if 'csc-csc' in run["lumisections"]:
            #print("Run :", r)
            data_dict = run["lumisections"]["csc-csc"]
            for key in data_dict.keys():
                if key == "GOOD":
                    dict["cscGOOD"] = data_dict["GOOD"]
                if key == "STANDBY":
                    dict["cscSTANDBY"] = data_dict["STANDBY"]
                if key == "BAD":
                    dict["cscBAD"] = data_dict["BAD"]
                if key == "EMPTY":
                    dict["cscEMPTY"] = data_dict["EMPTY"]
            del data_dict
        runreg_df = pd.concat([runreg_df, pd.DataFrame([dict])], ignore_index=True)
    except:
        dict = {"run_number": int(r), "class": "BAD", "cscGOOD": 0, "cscSTANDBY":0, "cscBAD":0, "cscEMPTY":0}
    if ("Collisions" not in dict["class"]) or dict["cscSTANDBY"]!=0 or dict["cscBAD"]!=0 or dict["cscGOOD"]==0:
        print("Run :", r, "--> BAD")
        bad_runs.append(int(r))
    else:
        print("Run :", r, "--> GOOD")
    print(dict)
    del dict
    del run

In [ ]:
print(bad_runs)

In [ ]:
monitoring_elements = monitoring_elements[~monitoring_elements['run_number'].isin(bad_runs)]

In [ ]:
run_list = np.sort(np.unique(monitoring_elements["run_number"].unique()))
print(run_list)

In [ ]:
with open("config.yaml", 'r') as f:
    try:
        info = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(f"Cannot read the file: {exc}")

In [ ]:
omsapi = OMSAPI("https://cmsoms.cern.ch/agg/api", "v1", cert_verify=False)
omsapi.auth_oidc(info["APIClient"]["client_ID"], info["APIClient"]["Client_Secret"])

In [ ]:
lumi_dfs = []
for r in run_list:
    print("Run :", r)
    df = []
    ls_query = omsapi.query("lumisections")
    ls_query.filter("run_number", r)
    ls_query.sort("lumisection_number", asc=False).paginate(page=1, per_page=100000)
    response = ls_query.data().json()["data"];
    for i in range(len(response)):
        df.append(response[i]["attributes"])
    lumi_dfs.append(pd.DataFrame(df)) # Downcasting object dtype arrays on .fillna is deprecated
    del df
    del response
    del ls_query

In [ ]:
for i in range(len(lumi_dfs)):
    lumi_dfs[i]['castor_ready'] = lumi_dfs[i]['castor_ready'].fillna(False)
    lumi_dfs[i]['gem_ready'] = lumi_dfs[i]['gem_ready'].fillna(False)
    lumi_dfs[i]['zdc_ready'] = lumi_dfs[i]['zdc_ready'].fillna(False)
    lumi_dfs[i]['prescale_index'] = lumi_dfs[i]['prescale_index'].fillna(-1)
    lumi_dfs[i]['prescale_name'] = lumi_dfs[i]['prescale_name'].fillna("")

In [ ]:
lumi_info = pd.concat(lumi_dfs)

In [ ]:
lumi_info = lumi_info.rename(columns={'lumisection_number': 'ls_number'})
lumi_info["mean_lumi"]=(lumi_info["init_lumi"]+lumi_info["end_lumi"])/2

In [ ]:
print(lumi_info.columns)

In [ ]:
monitoring_elements = pd.merge(monitoring_elements, lumi_info, on=['run_number', 'ls_number'], how='left') 

In [ ]:
#%memit

In [ ]:
monitoring_elements = monitoring_elements[(monitoring_elements["beams_stable"]==True) & (monitoring_elements["cscm_ready"]==True) & (monitoring_elements["cms_active"]==True) & (monitoring_elements["beam_present"]==True) & (monitoring_elements["physics_flag"]==True)]


In [ ]:
monitoring_elements = monitoring_elements.sort_values(by=['run_number', 'ls_number']).reset_index()
monitoring_elements = monitoring_elements.drop(columns=["index"])

In [ ]:
monitoring_elements

In [ ]:
monitoring_elements.to_parquet(me+'_'+out_label+'_s0.parquet', index=False)